In [1]:
import pandas as pd
import os
from datetime import datetime
import utils.data_utils as data_utils
from collections import defaultdict

LONG_TERM_ACTIONS = ['walking', 'eating', 'smoking', 'discussion']
ACTIONS = data_utils.define_actions('all')
ACTIONS_CMU = data_utils.define_actions_cmu('all')
ACTIONS_CMU.sort(key=len)

In [7]:
def csv_iter():
    for f in os.listdir('./checkpoint/test'):
#         if not f.startswith('5-5-10-10-10-15-15'):
#             continue
        try:
            datetime.strptime(f[-23:-4], "%d-%m-%Y-%H:%M:%S")
            if f.split('.')[-1] == 'csv':
                day = int(f[-23:-21])
                if day > 10:
                    yield f
        except:
            pass

def extend_df(df, long_term=True):
    if long_term:
        actions = LONG_TERM_ACTIONS
        time_lens = [560, 1000]
    else:
        actions = ACTIONS
        time_lens = [80, 160, 320, 400]
    columns_mapping = defaultdict(list)
    for time_len in time_lens:
        for action_name in actions:
            columns_mapping['3d' + str(time_len)].append(action_name + '3d' + str(time_len))
            columns_mapping['usingfulltestset_3d' + str(time_len)].append('usingfulltestset_' + action_name + '3d' + str(time_len))

    for key in columns_mapping:
        df[key] = df[columns_mapping[key]].mean(axis=1)
    
    return list(columns_mapping.keys())

def get_summary_df(merged):
    cols = list(set([i[:-5] for i in merged.columns]))
    new_cols = []
    for col in cols:
        col_name = [c for c in merged.columns if c.startswith(col)]
        new_cols.append(col+'_mean')
        new_cols.append(col+'_std')
        merged[col+'_mean'] = merged[col_name].mean(axis=1)
        merged[col+'_std'] = merged[col_name].std(axis=1)
    return merged[new_cols]

In [8]:
df_lst = []
mini_df_lst = []
for i, f in enumerate(csv_iter()):
    print(i, f)
    df = pd.read_csv(os.path.join('checkpoint/test', "./", f))
    df = df.sort_values(by='v_3d')
    df_lst.append(df[extend_df(df, LONG_TERM_ACTIONS)].add_suffix('_run' + str(i)))
    mini_df_lst.append(df[extend_df(df, LONG_TERM_ACTIONS)].head(1).add_suffix('_run' + str(i)).reset_index(drop=True))

merged = pd.concat(mini_df_lst, axis=1)

0 5-5-10-10-10-15-15main_3d_3D_in15_out25_dct_n_30_22-06-2020-16:17:29.csv
1 main_3d_3D_in10_out25_dct_n_30_17-06-2020-08:17:49.csv
2 5-5-10-10-10-15-15main_3d_3D_in15_out25_dct_n_30_23-06-2020-21:18:06.csv
3 5-5-10-10-10main_cmu_3d_3D_in10_out25_dct_n_30_27-06-2020-05:17:54.csv


Exception ignored in: <generator object csv_iter at 0x7fb2d908db50>
RuntimeError: generator ignored GeneratorExit


KeyError: 'v_3d'

In [47]:
get_summary_df(merged)

,usingfulltestset_3d560_mean,usingfulltestset_3d560_std,usingfulltestset_3d1000_mean,usingfulltestset_3d1000_std,3d1000_mean,3d1000_std,3d560_mean,3d560_std
0,59.312402,0.171246,79.850457,0.280009,66.508968,1.427072,55.718568,1.006279


In [48]:
mini_df_lst[0]

,3d560_run0,usingfulltestset_3d560_run0,3d1000_run0,usingfulltestset_3d1000_run0
0,56.14349,59.327828,66.855033,79.628547


In [49]:
mini_df_lst[1]

,3d560_run1,usingfulltestset_3d560_run1,3d1000_run1,usingfulltestset_3d1000_run1
0,54.224918,59.172015,65.360337,79.474911


In [50]:
mini_df_lst[2]

,3d560_run2,usingfulltestset_3d560_run2,3d1000_run2,usingfulltestset_3d1000_run2
0,56.904558,59.579918,66.913328,80.102856


In [51]:
mini_df_lst[3]

,3d560_run3,usingfulltestset_3d560_run3,3d1000_run3,usingfulltestset_3d1000_run3
0,55.995639,59.329523,68.50588,80.015809


In [52]:
mini_df_lst[4]

,3d560_run4,usingfulltestset_3d560_run4,3d1000_run4,usingfulltestset_3d1000_run4
0,55.324234,59.152728,64.910262,80.030161


In [ ]:
############################################ CMU #########################################################

In [2]:
def csv_iter():
    for f in os.listdir('./checkpoint/test'):
        if f.find('cmu') < 1:
            continue
        try:
            datetime.strptime(f[-23:-4], "%d-%m-%Y-%H:%M:%S")
            if f.split('.')[-1] == 'csv':
                day = int(f[-23:-21])
                if day > 10:
                    yield f
        except:
            pass

def extend_df(df, long_term=True):
#     if long_term:
#         actions = LONG_TERM_ACTIONS
#         time_lens = [560, 1000]
#     else:
#         actions = ACTIONS
#         time_lens = [80, 160, 320, 400]
    time_lens = [80, 160, 320, 400, 560, 1000]
    actions = ACTIONS_CMU
    columns_mapping = defaultdict(list)
    for time_len in time_lens:
        for action_name in actions:
            columns_mapping['3d' + str(time_len)].append(action_name + '3d' + str(time_len))
#             columns_mapping['usingfulltestset_3d' + str(time_len)].append('usingfulltestset_' + action_name + '3d' + str(time_len))

    for key in columns_mapping:
        df[key] = df[columns_mapping[key]].mean(axis=1)
    
    return list(columns_mapping.keys())

def get_summary_df(merged):
    cols = list(set([i[:-5] for i in merged.columns]))
    new_cols = []
    for col in cols:
        col_name = [c for c in merged.columns if c.startswith(col)]
        new_cols.append(col+'_mean')
        new_cols.append(col+'_std')
        merged[col+'_mean'] = merged[col_name].mean(axis=1)
        merged[col+'_std'] = merged[col_name].std(axis=1)
    return merged[new_cols]

In [3]:
df_lst = []
mini_df_lst = []
for i, f in enumerate(csv_iter()):
    print(i, f)
    df = pd.read_csv(os.path.join('checkpoint/test', "./", f))
    df = df.sort_values(by='t_l')
    df_lst.append(df[extend_df(df, LONG_TERM_ACTIONS)].add_suffix('_run' + str(i)))
    mini_df_lst.append(df[extend_df(df, LONG_TERM_ACTIONS)].head(1).add_suffix('_run' + str(i)).reset_index(drop=True))

merged = pd.concat(mini_df_lst, axis=1)

0 5-5-10-10-10main_cmu_3d_3D_in10_out25_dct_n_30_27-06-2020-05:17:54.csv
1 5-5-10-10-10main_cmu_3d_3D_in10_out25_dct_n_30_27-06-2020-00:56:35.csv
2 5-5-10-10-10main_cmu_3d_3D_in10_out25_dct_n_30_26-06-2020-22:41:45.csv
3 5-5-10-10-10main_cmu_3d_3D_in10_out25_dct_n_30_27-06-2020-03:07:28.csv
4 5-5-10-10-10main_cmu_3d_3D_in10_out25_dct_n_30_27-06-2020-07:27:49.csv


In [21]:
df_lst = []
mini_df_lst = []
for i, f in enumerate(csv_iter()):
    print(i, f)
    df = pd.read_csv(os.path.join('checkpoint/test', "./", f))
    df = df.sort_values(by='t_l')
    extend_df(df, LONG_TERM_ACTIONS)
    df = df[['basketball3d80', 'basketball3d160',
       'basketball3d320', 'basketball3d400', 'basketball3d560',
       'basketball3d1000', 'basketball_signal3d80', 'basketball_signal3d160',
       'basketball_signal3d320', 'basketball_signal3d400',
       'basketball_signal3d560', 'basketball_signal3d1000',
       'directing_traffic3d80', 'directing_traffic3d160',
       'directing_traffic3d320', 'directing_traffic3d400',
       'directing_traffic3d560', 'directing_traffic3d1000', 'jumping3d80',
       'jumping3d160', 'jumping3d320', 'jumping3d400', 'jumping3d560',
       'jumping3d1000', 'running3d80', 'running3d160', 'running3d320',
       'running3d400', 'running3d560', 'running3d1000', 'soccer3d80',
       'soccer3d160', 'soccer3d320', 'soccer3d400', 'soccer3d560',
       'soccer3d1000', 'walking3d80', 'walking3d160', 'walking3d320',
       'walking3d400', 'walking3d560', 'walking3d1000', 'washwindow3d80',
       'washwindow3d160', 'washwindow3d320', 'washwindow3d400',
       'washwindow3d560', 'washwindow3d1000', '3d80', '3d160', '3d320',
       '3d400', '3d560', '3d1000']]
    df = df.head(1).add_suffix('_run' + str(i)).reset_index(drop=True)
    mini_df_lst.append(df)

merged = pd.concat(mini_df_lst, axis=1)

0 5-5-10-10-10main_cmu_3d_3D_in10_out25_dct_n_30_27-06-2020-05:17:54.csv
1 5-5-10-10-10main_cmu_3d_3D_in10_out25_dct_n_30_27-06-2020-00:56:35.csv
2 5-5-10-10-10main_cmu_3d_3D_in10_out25_dct_n_30_26-06-2020-22:41:45.csv
3 5-5-10-10-10main_cmu_3d_3D_in10_out25_dct_n_30_27-06-2020-03:07:28.csv
4 5-5-10-10-10main_cmu_3d_3D_in10_out25_dct_n_30_27-06-2020-07:27:49.csv


In [22]:
merged

,basketball3d80_run0,basketball3d160_run0,basketball3d320_run0,basketball3d400_run0,basketball3d560_run0,basketball3d1000_run0,basketball_signal3d80_run0,basketball_signal3d160_run0,basketball_signal3d320_run0,basketball_signal3d400_run0,...,washwindow3d320_run4,washwindow3d400_run4,washwindow3d560_run4,washwindow3d1000_run4,3d80_run4,3d160_run4,3d320_run4,3d400_run4,3d560_run4,3d1000_run4
0,13.308311,23.384243,46.629196,58.205097,76.416893,107.87616,3.395906,6.495772,12.945606,16.926878,...,33.221855,43.872963,55.564011,77.489571,10.862974,19.379491,36.790796,46.252044,63.472875,96.995269


In [23]:
out = get_summary_df(merged)
out[sorted([i for i in out.columns if i[-4:] == 'mean'])]

,3d1000_mean,3d160_mean,3d320_mean,3d400_mean,3d560_mean,3d80_mean,basketball3d1000_mean,basketball3d160_mean,basketball3d320_mean,basketball3d400_mean,...,walking3d320_mean,walking3d400_mean,walking3d560_mean,walking3d80_mean,washwindow3d1000_mean,washwindow3d160_mean,washwindow3d320_mean,washwindow3d400_mean,washwindow3d560_mean,washwindow3d80_mean
0,95.706309,19.515544,36.997466,46.200378,62.713289,10.831285,102.04586,22.661903,44.699953,55.68073,...,19.953937,22.894094,27.885699,7.195072,80.47498,12.312403,32.165336,42.629682,54.679941,5.922688


In [24]:
sorted(ACTIONS_CMU)

['basketball',
 'basketball_signal',
 'directing_traffic',
 'jumping',
 'running',
 'soccer',
 'walking',
 'washwindow']

In [25]:
my_cols = []
for name in sorted(ACTIONS_CMU):
    my_cols.append(name + '3d80_mean')
    my_cols.append(name + '3d160_mean')
    my_cols.append(name + '3d320_mean')
    my_cols.append(name + '3d400_mean')
    my_cols.append(name + '3d1000_mean')
my_cols.append('3d80_mean')
my_cols.append('3d160_mean')
my_cols.append('3d320_mean')
my_cols.append('3d400_mean')
my_cols.append('3d1000_mean')
print(my_cols)

['basketball3d80_mean', 'basketball3d160_mean', 'basketball3d320_mean', 'basketball3d400_mean', 'basketball3d1000_mean', 'basketball_signal3d80_mean', 'basketball_signal3d160_mean', 'basketball_signal3d320_mean', 'basketball_signal3d400_mean', 'basketball_signal3d1000_mean', 'directing_traffic3d80_mean', 'directing_traffic3d160_mean', 'directing_traffic3d320_mean', 'directing_traffic3d400_mean', 'directing_traffic3d1000_mean', 'jumping3d80_mean', 'jumping3d160_mean', 'jumping3d320_mean', 'jumping3d400_mean', 'jumping3d1000_mean', 'running3d80_mean', 'running3d160_mean', 'running3d320_mean', 'running3d400_mean', 'running3d1000_mean', 'soccer3d80_mean', 'soccer3d160_mean', 'soccer3d320_mean', 'soccer3d400_mean', 'soccer3d1000_mean', 'walking3d80_mean', 'walking3d160_mean', 'walking3d320_mean', 'walking3d400_mean', 'walking3d1000_mean', 'washwindow3d80_mean', 'washwindow3d160_mean', 'washwindow3d320_mean', 'washwindow3d400_mean', 'washwindow3d1000_mean', '3d80_mean', '3d160_mean', '3d320_

In [35]:
values = [str(i).split('.')[0] + '.' + str(i).split('.')[1][:1] for i in list(out[my_cols].iloc[0])]
# print('&'.join(values[:20]))
# print()
# print('&'.join(values[20:]))
values[4::5]

['102.0', '57.0', '138.3', '163.5', '62.6', '122.1', '39.3', '80.4', '95.7']

In [18]:
from itertools import chain
sorted_cols = []
for actions in chain(ACTIONS_CMU, ['']):
    cols = []
    for time_len in [80, 160, 320, 400, 1000]:
        cols.append(actions + '3d' + str(time_len) + '_mean')
    print(out[cols])
    print('\n'*3)

   soccer3d80_mean  soccer3d160_mean  soccer3d320_mean  soccer3d400_mean  \
0         11.23921         22.198215         45.166055         58.181467   

   soccer3d1000_mean  
0         122.171361  




   jumping3d80_mean  jumping3d160_mean  jumping3d320_mean  jumping3d400_mean  \
0         14.857477          31.113573          71.207083          91.350748   

   jumping3d1000_mean  
0          163.593665  




   running3d80_mean  running3d160_mean  running3d320_mean  running3d400_mean  \
0          24.51249          37.010357            39.9759          41.910265   

   running3d1000_mean  
0           62.642335  




   walking3d80_mean  walking3d160_mean  walking3d320_mean  walking3d400_mean  \
0          7.195072          11.109052          19.953937          22.894094   

   walking3d1000_mean  
0           39.348861  




   basketball3d80_mean  basketball3d160_mean  basketball3d320_mean  \
0            12.763901             22.661903             44.699953   

   basketball3d40

In [62]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

In [36]:
with open('cmu.txt', 'r') as f:
    counter = 0
    for i, line in enumerate(f):
        if len(line) > 5:
            tmp = line.split(']')[-1]
            tmp = tmp.strip('\n')
            tmp = tmp.strip(' ')
            print(tmp.split(' '))
            counter += 1
            if counter == 3:
                print()
                print()
                counter = 0

['18.4', '33.8', '59.5', '70.5', '106.7', '12.7', '23.8', '40.3', '46.7', '77.5', '15.2', '29.6', '55.1', '66.1', '127.1', '36.0', '68.7', '125.0', '145.5', '195.5', '15.6', '19.4', '31.2', '36.2', '43.3']
['16.7', '30.5', '53.8', '64.3', '91.5', '8.4', '16.2', '30.8', '37.8', '76.5', '10.6', '20.3', '38.7', '48.4', '115.5', '22.4', '44.0', '87.5', '106.3', '162.6', '14.3', '16.3', '18.0', '20.2', '27.5']
['14.0', '25.4', '49.6', '61.4', '106.1', '3.5', '6.1', '11.7', '15.2', '53.9', '7.4', '15.1', '31.7', '42.2', '152.4', '16.9', '34.4', '76.3', '96.8', '164.6', '25.5', '36.7', '39.3', '39.9', '58.2']


['20.3', '39.5', '71.3', '84', '129.6', '8.2', '13.7', '21.9', '24.5', '32.2', '8.4', '15.8', '29.3', '35.4', '61.1', '16.8', '30.5', '54.2', '63.6', '77.8']
['12.1', '21.8', '41.9', '52.9', '94.6', '7.6', '12.5', '23.0', '27.5', '49.8', '8.2', '15.9', '32.1', '39.9', '58.9', '12.5', '22.2', '40.7', '49.7', '63.4']
['11.3', '21.5', '44.2', '55.8', '117.5', '7.7', '11.8', '19.4', '23.1'

In [45]:
resup = [float(i) for i in ['18.4', '33.8', '59.5', '70.5', '106.7', '12.7', '23.8', '40.3', '46.7', '77.5', '15.2', '29.6', '55.1', '66.1', '127.1', '36.0', '68.7', '125.0', '145.5', '195.5', '15.6', '19.4', '31.2', '36.2', '43.3'] + ['20.3', '39.5', '71.3', '84', '129.6', '8.2', '13.7', '21.9', '24.5', '32.2', '8.4', '15.8', '29.3', '35.4', '61.1']]
assert(len(resup[4::5]) == 8)
np.array(resup[4::5]).mean()

96.625

In [42]:
import numpy as np

In [47]:
convseq = [float(i) for i in ['16.7', '30.5', '53.8', '64.3', '91.5', '8.4', '16.2', '30.8', '37.8', '76.5', '10.6', '20.3', '38.7', '48.4', '115.5', '22.4', '44.0', '87.5', '106.3', '162.6', '14.3', '16.3', '18.0', '20.2', '27.5'] + ['12.1', '21.8', '41.9', '52.9', '94.6', '7.6', '12.5', '23.0', '27.5', '49.8', '8.2', '15.9', '32.1', '39.9', '58.9']]
assert(len(convseq[4::5]) == 8)
np.array(convseq[4::5]).mean()

84.6125

In [52]:
np.array(convseq[4::5])

array([ 91.5,  76.5, 115.5, 162.6,  27.5,  94.6,  49.8,  58.9])

In [63]:
import itertools

lst = []
for len_ in range(1, 8):
    for i in itertools.combinations(np.array(resup[4::5]), len_):
        lst.append(sum(i)/len_)
sorted(lst)

[32.2,
 37.75,
 43.3,
 45.53333333333333,
 46.650000000000006,
 51.0,
 52.2,
 53.525,
 54.85,
 56.93333333333334,
 60.4,
 60.63333333333333,
 60.73333333333333,
 60.824999999999996,
 61.1,
 64.16,
 64.925,
 65.925,
 66.55,
 66.66666666666667,
 67.53333333333332,
 68.24,
 68.36666666666666,
 68.74,
 69.3,
 69.375,
 69.45,
 70.02499999999999,
 70.36666666666666,
 70.64999999999999,
 72.13333333333333,
 72.15,
 73.46666666666667,
 74.08000000000001,
 74.3,
 74.47500000000001,
 74.58000000000001,
 74.64999999999999,
 75.0,
 75.06666666666668,
 75.10000000000001,
 75.83333333333333,
 77.16666666666666,
 77.25,
 77.325,
 77.35999999999999,
 77.5,
 77.86,
 77.875,
 77.95,
 77.99999999999999,
 78.46666666666667,
 78.66,
 78.93333333333334,
 79.65,
 79.76666666666667,
 80.9,
 80.91999999999999,
 81.41999999999999,
 81.76666666666667,
 81.775,
 81.92,
 81.94,
 82.4,
 82.5,
 82.63333333333333,
 83.025,
 83.05,
 83.14,
 83.33333333333334,
 83.46666666666665,
 83.64000000000001,
 83.9,
 84.55000000